In [3]:
import numpy as np

class PNN:
    """ 
        Class to represent the autoencoder and reflect the customizable pattern. 

        Args: 
            X_train (numpy array) : X data. 
            y_train (numpy array) : y data.
            imbalance_handler (sampler) : Imbalance handler object.
            kernel (str) : Type of the kernel.
            bandwidth (float): Size of the bandwidth. 
    """
    def __init__(self, X_train, y_train, imbalance_handler, kernel = None, bandwidth = 0.1):

        self.X_train = X_train
        self.y_train = y_train
        X_res, y_res = imbalance_handler.resampling(self.X_train, self.y_train)
    
        self.X_res = X_res
        self.y_res = y_res
        self.kernel = kernel
        self.bandwidth = bandwidth
        

    def kernel_functions(self):
        """ 
        Kernel functions to be used in PNN. 
        
            Returns: 
                Returns the function depending on the kernel type.
        """
        if self.kernel == "uniform": 
            
            return lambda x: 0.5 if np.abs(x/self.bandwidth) <= 1 else 0
            
            
        elif self.kernel == "triangular": 

            return lambda x: (1 - np.abs(x/self.bandwidth)) if np.abs(x/self.bandwidth) <= 1 else 0
           
            
        elif self.kernel == "epanechnikow": 

            return lambda x: 0.75*(1 - (x/self.bandwidth)**2) if np.abs(x/self.bandwidth) <= 1 else 0
            
            
        elif self.kernel == "gaussian": 

            return lambda x: (1/((2*np.pi)**0.5))*np.exp(-0.5*(x/self.bandwidth)**2)
           

            
    def pattern_layer(self, x):
        """ 
        Pattern layer to apply kernel values.
        
            Parameters: 
                x (array) : Training array of x.  
        
            Returns: 
                Returns values as array after kernel function implementation.
        """
        
        x_arr = np.array(x, dtype=float)
        euclidean_dist = np.linalg.norm(self.X_res - x_arr, axis = 1)
        values = []
        kernel_values = self.kernel_functions()
        for d in euclidean_dist: 
            values.append(kernel_values(d))

        values_arr = np.array(values)
        
        return values_arr 

    def summation_layer(self, values_arr):

        """ 
        Summation layer to calculate the average.
        
            Parameters: 
                values_arr (array) : Array from pattern layer.  
        
            Returns: 
                Returns values as list after summation.
        """
        y_arr = np.array(self.y_res)
        
        sum_values_0 = values_arr[y_arr==0].sum()
        sum_values_1 = values_arr[y_arr==1].sum()

        layer_values = [sum_values_0, sum_values_1]

        average_values_0 = sum_values_0 / (self.y_res.value_counts()[0])
        average_values_1 = sum_values_1 / (self.y_res.value_counts()[1])

        layer_val_avg = [average_values_0, average_values_1]
        return layer_val_avg

    def output_layer(self, layer_values):
        """ 
        Output layer to calculate the probabilities with considering prior probabilities (Applying Bayesian).
        
            Parameters: 
                layer_values (array) : List from summation layer.  
        
            Returns: 
                Returns the max value.
        """
        prior_prob0 = len(self.y_res[self.y_res==0])/len(self.y_res)
        prior_prob1 = len(self.y_res[self.y_res==1])/len(self.y_res)

        posterior_score0 = layer_values[0]*prior_prob0
        posterior_score1 = layer_values[1]*prior_prob1

        max_value = np.argmax([posterior_score0, posterior_score1])
        
        
        return max_value


    def pnn_pred(self, input_test):
        """ 
        Final layer to use each layer together.
        
            Parameters: 
                input_test (array) : Array for testing.  
        
            Returns: 
                Returns labels.
        """
        input_test_arr = np.array(input_test)
        labels = []
        for i in input_test_arr:

            k = self.pattern_layer(i)
            s = self.summation_layer(k)
            o = self.output_layer(s)

            labels.append(int(o))

        return labels

    def analysis(self, y_true, X_test):

        """ 
        Analysis to check the performance of PNN.
        
            Parameters: 
                y_true (array) : Array for true values of y.
                X_test (array) : Test array. 

            Returns: 
                Returns confusion matrix.
        """
        labels = self.pnn_pred(X_test)
        labels_arr = np.array(labels)
        
        cm = confusion_matrix(y_true, labels_arr)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d")
        plt.title("Confusion Matrix")
        plt.ylabel("Actual Class")
        plt.xlabel("Predicted Class")

        plt.show()
            

## References

1- Medium whyamit101. *Understanding Euclidean distance with NumPy*. Accessed on April 16, 2025, from https://medium.com/@whyamit101/understanding-euclidean-distance-with-numpy-3fe9949ff196